In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from FoodSeg103_data_handler import *

c:\Users\roman\anaconda3\envs\ilastik2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create scribbles

Load the ground truths as batch

In [7]:
img_nums = [n for n in range(0, 4900, 800)] #[1382] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Total images: 7


Check and filter for resolution

In [8]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 250_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

Resolutions:
   0:   196,608 pixels
 800:   262,144 pixels (!)
1600:   273,920 pixels (!)
2400:   252,970 pixels (!)
3200:   750,000 pixels (!)
4000:   196,608 pixels
4800:    77,120 pixels


In [9]:
# Filter out those images
gts = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")

Images with resolution under 250,000: 3
   0: 196,608 pixels
4000: 196,608 pixels
4800:  77,120 pixels


Define scribbles parameters

In [11]:
# Which scribbles to create
bins = [0.06, 0.3, 1.45]
suff = ["w3"] #standing for width 3
sq_scaling = 100
scribble_width = 3
scribbles_seed = 1

# Where to save the scribbles
folder_path = "./FoodSeg103_results/data/run02"

save_res = False
show_res = False
print_steps = True

Loop and create scribbles

In [12]:
percentages = {}
for bin in bins:
    percentages[bin] = []
    for img_num, gt in gts.items():
        for s in suff:
            np.random.seed(scribbles_seed)
            print(f"IMG {img_num}: bin {bin}, suff {s}")
            scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, margin=0.75, sq_scaling=sq_scaling, mode="all",
                                                            save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps, scribble_width=scribble_width)
            percentages[bin].append(perc_labelled)
            print(f"Annotation percentage: {perc_labelled:2.2f}%\n")

IMG 0: bin 0.06, suff w3
CLASS 1:
sk_max_pix: 9.45, sq_size: 44, sk_pix_range: (9, 88)
Adjusting square size and range to 22 (9, 88)
Adjusting square size and range to 11 (5, 88)
Adjusting square size and range to 5 (2, 88)
   prim_sk_squares pix: 15 = 0.03%
Adjusting square size and range to 22 (9, 88)
Adjusting square size and range to 11 (5, 88)
   sec_sk_squares pix: 7 = 0.01%
lines_max_pix: 9.45, line_pix_range: (9, 88)
   lines pix: 9 = 0.02%
TOTAL pix: 31 = 0.07%
CLASS 67:
sk_max_pix: 14.65, sq_size: 44, sk_pix_range: (14, 88)
Adjusting square size and range to 22 (11, 88)
   prim_sk_squares pix: 12 = 0.02%
Adjusting square size and range to 22 (11, 88)
   sec_sk_squares pix: 12 = 0.02%
lines_max_pix: 14.65, line_pix_range: (14, 88)
Adjusting line range to (7, 176)
   lines pix: 14 = 0.02%
TOTAL pix: 38 = 0.05%
CLASS 91:
sk_max_pix: 5.82, sq_size: 44, sk_pix_range: (5, 88)
Adjusting square size and range to 22 (5, 88)
Adjusting square size and range to 11 (5, 88)
Adjusting squar

In [ ]:
np.sum(scribbles>0) / np.prod(scribbles.shape)

Quick report of labelling percentages:

In [13]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

labelled percentage in bin 0.06: mean = 0.2737, std = 0.0162, range = 0.2518 - 0.2905
labelled percentage in bin 0.3: mean = 1.1885, std = 0.1522, range = 0.9751 - 1.3194
labelled percentage in bin 1.45: mean = 5.0873, std = 0.2270, range = 4.7886 - 5.3385


Show the last created scribble with its ground truth, and all ground truths (optionally)

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")